# Hypothesis Test using NHANES data

In [1]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg') # workaround, there may be a better way
import statsmodels.api as sm
import scipy.stats.distributions as dist

In [2]:
url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

In [3]:
da["SMQ020x"] = da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210,SMQ020x,RIAGENDRx
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0,Yes,Male
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN,Yes,Male
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0,Yes,Male
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0,No,Female
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0,No,Female


## 1. Hypothesis Tests for One Proportion

어느 나라의 흡연자 비율이 40%라고 한다.   
   
NHANES 데이터를 활용해서 미국의 흡연자의 비율이 40%와 같은지 다른지 비교해보려고 한다.   
 
귀무가설 = 미국의 흡연자 비율은 40%이다. p = pnull(0.4)   
대립가설 = 미국의 흡연자 비율은 40%가 아니다. p != pnull(0.4)

In [4]:
pnull = 0.4

x = da.SMQ020x.dropna() == 'Yes'
p = x.mean()
p

0.4050655021834061

In [5]:
se = np.sqrt(pnull*(1-pnull)/len(x))
se

0.00647467353462031

In [6]:
test_stat = (p-0.4)/se
test_stat

0.7823563854332805

In [7]:
pvalue = 2 * dist.norm.cdf(-np.abs(test_stat))
pvalue

0.4340051581348052

#### statsmodel로 계산

In [8]:
sm.stats.proportions_ztest(p * len(x), len(x), pnull, alternative='two-sided')

(0.7807518954896244, 0.43494843171868214)

### 해석

pvalue가 유의 수준 0.05보다 훨씬 크다.   
pvalue가 크기 때문에 귀무가설을 기각할 수 없다.   
미국의 흡연자 비율은 40%로 추정된다.   

# 

## 2. Hypothesis Tests for Two Proportions

비교 테스트는 모집단을 고정 값과 비교하는 테스트보다 훨씬 더 자주 사용된다.      
   
NHANES 데이터를 활용해서 미국의 남성과 여성의 흡연자의 비율을 비교해보려고 한다.   
나이가 20~25세의 그룹에서 수행하려고 한다.   

귀무가설 = 미국의 남성과 여성의 흡연자 비율은 같다. p1 - p2 = 0   
대립가설 = 미국의 남성과 여성의 흡연자 비율은 다르다. p1 - p2 != 0

In [9]:
dx = da[["SMQ020x", "RIDAGEYR", "RIAGENDRx"]].dropna()

dx.head()

,SMQ020x,RIDAGEYR,RIAGENDRx
0,Yes,62,Male
1,Yes,53,Male
2,Yes,78,Male
3,No,56,Female
4,No,42,Female


In [10]:
dx = dx[dx['RIDAGEYR']>=20]
dx = dx[dx['RIDAGEYR']<=25]
dx

,SMQ020x,RIDAGEYR,RIAGENDRx
6,Yes,22,Male
17,No,24,Female
26,Yes,22,Male
38,No,20,Female
40,Yes,24,Male
...,...,...,...
5688,No,25,Male
5701,No,25,Male
5707,No,25,Female
5729,No,25,Male


In [11]:
p = dx.groupby("RIAGENDRx")["SMQ020x"].agg([lambda z: np.mean(z == "Yes"), "size"])
p.columns = ["Smoke", "N"]
p

,Smoke,N
RIAGENDRx,,
Female,0.238971,272
Male,0.341270,252


In [12]:
p_comb = (dx.SMQ020x == "Yes").mean()
va = p_comb * (1 - p_comb)

se = np.sqrt(va * (1 / p.N.Female + 1 / p.N.Male))
(p_comb, va, se)

(0.2881679389312977, 0.2051271779033856, 0.039599757248262944)

In [13]:
test_stat = (p.Smoke.Female - p.Smoke.Male) / se
p_value = 2 * dist.norm.cdf(-np.abs(test_stat))
(test_stat, p_value)

(-2.5833303066279414, 0.009785159057508375)

#### statsmodel로 계산

In [14]:
dx_females = dx.loc[dx.RIAGENDRx == "Female", "SMQ020x"].replace({"Yes": 1, "No": 0})
dx_females.head()

17     0
38     0
46     0
69     1
102    1
Name: SMQ020x, dtype: int64

In [15]:
dx_males = dx.loc[dx.RIAGENDRx == "Male", "SMQ020x"].replace({"Yes": 1, "No": 0})
dx_males.head()

6     1
26    1
40    1
48    0
96    0
Name: SMQ020x, dtype: int64

In [16]:
count = np.array([len(dx_females[dx_females==1]), len(dx_males[dx_males==1])])
count

array([65, 86])

In [17]:
nobs = np.array([len(dx_females), len(dx_males)])
nobs

array([272, 252])

In [18]:
sm.stats.proportions_ztest(count,nobs, alternative='two-sided')

(-2.5833303066279414, 0.009785159057508375)

### 해석

pvalue가 유의 수준 0.05보다 작다.   
pvalue가 작기 때문에 귀무가설을 기각할 수 있다.     
귀무가설을 기각하고 대립가설을 지지한다.   
미국의 남성과 여성의 흡연자 비율은 다를 것으로 추정한다.

# 

## 3. Hypothesis Tests Comparing Means

평균 검정은 비율 검정과 여러 면에서 유사하다.   
   
### 1) 
NHANES 데이터를 통해 평균 혈압이 고정된 수치(120)와 비교하여 통계적으로 유의한 차이가 있는지 확인해보자.   
40~50세의 남성들을 대상으로 수행하려고 한다.   

Hnull: 40 ~ 50세 남성의 평균 혈압은 120이다. (평균 - 120 = 0)   
Ha: 40 ~ 50세 남성의 평균 혈압은 120이 아니다. (평균 - 120 != 0)   

In [19]:
dx = da[["BPXSY1", "RIDAGEYR", "RIAGENDRx"]].dropna()
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDRx
0,128.0,62,Male
1,146.0,53,Male
2,138.0,78,Male
3,132.0,56,Female
4,100.0,42,Female


In [20]:
dx = dx.loc[(dx.RIDAGEYR >= 40) & (dx.RIDAGEYR <= 50) & (dx.RIAGENDRx == "Male"), :]
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDRx
10,144.0,46,Male
11,116.0,45,Male
20,110.0,49,Male
42,128.0,42,Male
51,118.0,50,Male


In [21]:
mean = dx.BPXSY1.mean()
mean

125.86698337292161

In [22]:
std = dx.BPXSY1.std()
std

16.115694145244138

In [23]:
se = std/np.sqrt(len(dx))
se

0.7854308737514324

In [24]:
test_stat = (mean-120)/se
test_stat

7.4697641371026

In [25]:
pvalue = 2 * dist.norm.cdf(-np.abs(test_stat))
pvalue

8.033869113167759e-14

#### statsmodel로 계산

In [26]:
sm.stats.ztest(dx.BPXSY1, value=120)

(7.469764137102597, 8.033869113167905e-14)

### 해석

pvalue가 유의 수준 0.05보다 작다. 거의 0에 가깝다.    
pvalue가 매우 작기 때문에 귀무가설을 기각할 수 있다.     
귀무가설을 기각하고 대립가설을 지지한다.   
40~50세 남성의 평균 혈압은 120이 아닐 것으로 추정한다.

## 

### 2)
귀무가설: 50-60세 남성의 평균 혈압과 50-60세 여성의 평균 혈압은 같다.   
대립가설: 50-60세 남성의 평균 혈압과 50-60세 여성의 평균 혈압은 다르다.

In [27]:
dx = da[["BPXSY1", "RIDAGEYR", "RIAGENDRx"]].dropna()
dx = dx.loc[(dx.RIDAGEYR >= 50) & (dx.RIDAGEYR <= 60), :]
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDRx
1,146.0,53,Male
3,132.0,56,Female
9,178.0,56,Male
15,134.0,57,Female
19,136.0,54,Female


In [28]:
bpx_female = dx.loc[dx.RIAGENDRx=="Female", "BPXSY1"]
bpx_male = dx.loc[dx.RIAGENDRx=="Male", "BPXSY1"]


In [29]:
female_mean = bpx_female.mean()
female_std = bpx_female.std()
male_mean = bpx_male.mean()
male_std = bpx_male.std()
(female_mean,female_std,male_mean,male_std)

(127.92561983471074,
 18.388341472234025,
 129.23829787234044,
 18.283441810319484)

In [30]:
se = np.sqrt((female_std**2/len(bpx_female) + (male_std**2/len(bpx_male))))
se

1.1873756999377034

In [31]:
t_test = (female_mean - male_mean)/se
t_test

-1.1055288041506708

In [32]:
pvalue = 2 * dist.norm.cdf(-np.abs(t_test))
pvalue

0.2689305203990455

#### statsmodel로 계산

In [33]:
sm.stats.ttest_ind(bpx_female, bpx_male)        # test statistic, pvalue, degree of freedom

(-1.105435895556249, 0.26925004137768577, 952.0)

### 해석

pvalue가 유의 수준 0.05보다 크다.    
pvalue가 0.05보다 크기 때문에 귀무가설을 기각할 수 없다.     
귀무가설을 지지한다.   
50-60세 남성의 평균 혈압과 50-60세 여성의 평균 혈압의 차이는 유의미하지 않을 것으로 추정한다.

# 

### 3) Paired test

예를 들어 NHANES 데이터에서 혈압은 최소 두번 측정된다. 똑같이 반복하더라도 두번의 측정에서 같은 수치가 나오지 않는다(두번째 측정에서 사람들은 더 긴장을 하기 때문에 낮게 측정된다.). 이와 같이 paired test는 같은 객체에서 반복 측정한 수치들에 대한 mean t-test이다.   
   
paired test는 첫번째 측정과 두번째 측정의 평균의 차이를 0과 비교하는 one sample t-test와 동일하다.   
   
귀무가설 : 첫번째 혈압측정과 두번째 혈압측정은 차이가 없다. ( mean(n1 - n2) = 0 )   
대립가설 : 첫번째 혈압측정과 두번째 혈압측정은 차이가 있다. ( mean(n1 - n2) != 0 ) 

In [34]:
dx = da[["BPXSY1", "BPXSY2"]].dropna()
db = dx.BPXSY1 - dx.BPXSY2

In [35]:
mean = db.mean()
std = db.std()
(mean, std)

(0.6749860309182343, 5.046465367896533)

In [36]:
se = std/np.sqrt(len(db))
se

0.06887166703842532

In [37]:
t_test = mean/se
t_test

9.800634425498105

In [38]:
pvalue = 2 * dist.norm.cdf(-np.abs(t_test))
pvalue

1.1188070930942068e-22

#### statsmodel로 계산

In [39]:
sm.stats.ztest(db)

(9.800634425497911, 1.1188070930963587e-22)

### 해석
   
pvalue가 유의 수준 0.05보다 작다. 거의 0에 가깝다.    
pvalue가 매우 작기 때문에 귀무가설을 기각할 수 있다.     
귀무가설을 기각하고 대립가설을 지지한다.   
첫번째 혈압측정과 두번째 혈압측정의 유의미한 차이가 있다고 추정한다.

# 

### 4) 나이별, 성별로 paird test

In [40]:
dx = da[["RIAGENDRx", "BPXSY1", "BPXSY2", "RIDAGEYR"]].dropna()
dx["agegrp"] = pd.cut(dx.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80])

In [41]:
dx.head()

,RIAGENDRx,BPXSY1,BPXSY2,RIDAGEYR,agegrp
0,Male,128.0,124.0,62,"(60, 70]"
1,Male,146.0,140.0,53,"(50, 60]"
2,Male,138.0,132.0,78,"(70, 80]"
3,Female,132.0,134.0,56,"(50, 60]"
4,Female,100.0,114.0,42,"(40, 50]"


In [42]:
dx["agegrp"] = pd.cut(dx.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80])
for k, g in dx.groupby(["RIAGENDRx", "agegrp"]):
    db = g.BPXSY1 - g.BPXSY2
    print(k, db.mean(), db.size, sm.stats.ztest(db))

('Female', Interval(18, 30, closed='right')) 0.13708260105448156 569 (0.7612107360791227, 0.4465312067051751)
('Female', Interval(30, 40, closed='right')) 0.6713615023474179 426 (3.307398751951031, 0.0009416674523368051)
('Female', Interval(40, 50, closed='right')) 0.5970149253731343 469 (2.6040611621024654, 0.009212631487347644)
('Female', Interval(50, 60, closed='right')) 0.7685393258426966 445 (3.1023718750881724, 0.001919766301204196)
('Female', Interval(60, 70, closed='right')) 0.8787878787878788 396 (3.1024528501809625, 0.0019192411825181255)
('Female', Interval(70, 80, closed='right')) 1.4512820512820512 390 (5.141706875154317, 2.722536503552981e-07)
('Male', Interval(18, 30, closed='right')) 0.00390625 512 (0.01959622841647691, 0.9843654725443948)
('Male', Interval(30, 40, closed='right')) 0.46296296296296297 432 (1.9451535788714596, 0.05175649697939119)
('Male', Interval(40, 50, closed='right')) 0.17894736842105263 380 (0.7201800810138878, 0.47141412641258706)
('Male', Interva

### 해석
여성의 경우, 18-30세 그룹을 제외하고는 첫번째 혈압측정과 두번째 혈압측정의 유의미한 차이를 나타냈다.      
반면 남성의 경우, 60-80세의 나이가 많은 그룹만이 첫번째 혈압측정과 두번째 혈압측정의 유의미한 차이를 나타냈다.   

나이가 들수록 혈압 측정의 차이는 유의미한 차이를 나타낸다.   
나이가 많은 그룹에서는 여성보다 남성이 더 뚜렷하게 유의미한 차이를 나타낸다.   